In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext.getOrCreate()
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
sqlContext = SQLContext(sc)

In [2]:
node = sqlContext.read.load("Data/hungary.osm.pbf.node.parquet")
way = sqlContext.read.load("Data/hungary.osm.pbf.way.parquet")
relations = sqlContext.read.load("Data/hungary.osm.pbf.relation.parquet")


In [3]:
from pyspark.sql.functions import size
node.filter(size(node.tags) != 0).count()

702291

In [4]:
node.count()

18235204

In [5]:
from pyspark.sql.functions import size
filtered = node.filter(size(node.tags) != 0)

In [6]:
# from pyspark.sql.functions import array_contains
# filtered.where(array_contains("tags", "highway"))
# correct but not correct type

from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf 

def exists(f):
    return udf(lambda xs: any(f(x) for x in xs), BooleanType())

# df.where(exists(lambda x: x > 3)("v"))

In [7]:
yy = (filtered.take(6))[0]


In [8]:
yy.tags[0].key.decode()

'highway'

In [9]:
node.select('tags').distinct().head()

Row(tags=[Row(key=bytearray(b'is_in'), value=bytearray(b'P\xc3\xa9csi kist\xc3\xa9rs\xc3\xa9g; Baranya megye; D\xc3\xa9l-Dun\xc3\xa1nt\xc3\xbal; Magyarorsz\xc3\xa1g')), Row(key=bytearray(b'is_in:country'), value=bytearray(b'Hungary')), Row(key=bytearray(b'is_in:county'), value=bytearray(b'Baranya')), Row(key=bytearray(b'is_in:subregion'), value=bytearray(b'P\xc3\xa9csi')), Row(key=bytearray(b'ksh_ref'), value=bytearray(b'32151')), Row(key=bytearray(b'name'), value=bytearray(b'Bog\xc3\xa1d')), Row(key=bytearray(b'place'), value=bytearray(b'village')), Row(key=bytearray(b'population'), value=bytearray(b'989')), Row(key=bytearray(b'postal_code'), value=bytearray(b'7742')), Row(key=bytearray(b'source:population'), value=bytearray(b'KSH (www.ksh.hu)')), Row(key=bytearray(b'source:population:date'), value=bytearray(b'2016-01-01')), Row(key=bytearray(b'source:postal_code'), value=bytearray(b'Magyar Posta (posta.hu)')), Row(key=bytearray(b'source:postal_code:date'), value=bytearray(b'2009-03-0

In [10]:
from pyspark.sql.functions import size
xx = node.select('tags').filter(size(node.tags)!=0)

In [11]:
l1 = xx.take(100)

In [12]:
l1[1]

Row(tags=[Row(key=bytearray(b'name'), value=bytearray(b'M\xc3\xb6rbisch am See')), Row(key=bytearray(b'traffic_sign'), value=bytearray(b'city_limit'))])

In [13]:
l1[1][0][0]

Row(key=bytearray(b'name'), value=bytearray(b'M\xc3\xb6rbisch am See'))

In [14]:
%matplotlib inline
import osmnx as ox

In [15]:
# import osmnx as ox
# import networkx as nx
# import geopandas as gpd
# import matplotlib.pyplot as plt
# import pandas as pd

# # place_name = "Hungary"
# # # graph = ox.graph_from_place(place_name, network_type='drive')
# # # fig, ax = ox.plot_graph(graph)

In [16]:
# ox.graph_from_file('Data/hungary.osm')

In [17]:
xx = way.take(2)

In [18]:
xx[0]

Row(id=3175810, version=7, timestamp=1531865296000, changeset=60814610, uid=408450, user_sid=bytearray(b'B\xc3\xa1thoryP\xc3\xa9ter'), tags=[Row(key=bytearray(b'highway'), value=bytearray(b'residential')), Row(key=bytearray(b'name'), value=bytearray(b'Honv\xc3\xa9d utca'))], nodes=[Row(index=0, nodeId=15231786), Row(index=1, nodeId=1310148452), Row(index=2, nodeId=1310021025), Row(index=3, nodeId=1310020985), Row(index=4, nodeId=5766745506), Row(index=5, nodeId=1310020988), Row(index=6, nodeId=1310021030), Row(index=7, nodeId=1310021038), Row(index=8, nodeId=1310021044), Row(index=9, nodeId=1309949162), Row(index=10, nodeId=1309949099), Row(index=11, nodeId=1282709263), Row(index=12, nodeId=5766742402), Row(index=13, nodeId=1234760030)])

In [19]:
xx[1]

Row(id=3175943, version=3, timestamp=1366287135000, changeset=15772683, uid=408450, user_sid=bytearray(b'B\xc3\xa1thoryP\xc3\xa9ter'), tags=[Row(key=bytearray(b'highway'), value=bytearray(b'residential')), Row(key=bytearray(b'name'), value=bytearray(b'Szent Istv\xc3\xa1n utca'))], nodes=[Row(index=0, nodeId=15234255), Row(index=1, nodeId=15231786)])

In [20]:
def extract_nodeids(way_row):
    node_list = []
    for x in way_row.nodes:
        node_list.append(x.nodeId)
    return node_list
        

In [21]:
nr = extract_nodeids(xx[0])
print (nr)

[15231786, 1310148452, 1310021025, 1310020985, 5766745506, 1310020988, 1310021030, 1310021038, 1310021044, 1309949162, 1309949099, 1282709263, 5766742402, 1234760030]


In [22]:
# node.filter(node.id in nr).collect()
# df.filter(df.age > 3).collect()
#  df[df.name.isin("Bob", "Mike")].collect()
yy = node[node.id.isin(nr)].collect()

In [23]:
zz = node.filter((node.id.isin(nr))).collect()

In [24]:
xx = way.take(2)

In [25]:
xx[0]

Row(id=3175810, version=7, timestamp=1531865296000, changeset=60814610, uid=408450, user_sid=bytearray(b'B\xc3\xa1thoryP\xc3\xa9ter'), tags=[Row(key=bytearray(b'highway'), value=bytearray(b'residential')), Row(key=bytearray(b'name'), value=bytearray(b'Honv\xc3\xa9d utca'))], nodes=[Row(index=0, nodeId=15231786), Row(index=1, nodeId=1310148452), Row(index=2, nodeId=1310021025), Row(index=3, nodeId=1310020985), Row(index=4, nodeId=5766745506), Row(index=5, nodeId=1310020988), Row(index=6, nodeId=1310021030), Row(index=7, nodeId=1310021038), Row(index=8, nodeId=1310021044), Row(index=9, nodeId=1309949162), Row(index=10, nodeId=1309949099), Row(index=11, nodeId=1282709263), Row(index=12, nodeId=5766742402), Row(index=13, nodeId=1234760030)])

In [26]:
way.printSchema()

root
 |-- id: long (nullable = true)
 |-- version: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- changeset: long (nullable = true)
 |-- uid: integer (nullable = true)
 |-- user_sid: binary (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: binary (nullable = true)
 |    |    |-- value: binary (nullable = true)
 |-- nodes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- index: integer (nullable = true)
 |    |    |-- nodeId: long (nullable = true)



In [27]:
xx[1].tags

[Row(key=bytearray(b'highway'), value=bytearray(b'residential')),
 Row(key=bytearray(b'name'), value=bytearray(b'Szent Istv\xc3\xa1n utca'))]

In [28]:
ex1 = node.select('tags').distinct()

In [29]:
ex2 = ex1.take(10)

In [30]:
node.printSchema()

root
 |-- id: long (nullable = true)
 |-- version: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- changeset: long (nullable = true)
 |-- uid: integer (nullable = true)
 |-- user_sid: binary (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: binary (nullable = true)
 |    |    |-- value: binary (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [31]:
node_nonempty = node.filter(size('tags') > 0)

In [32]:
node_nonempty.take(1)

[Row(id=277309, version=11, timestamp=1368049401000, changeset=16038871, uid=408450, user_sid=bytearray(b'B\xc3\xa1thoryP\xc3\xa9ter'), tags=[Row(key=bytearray(b'highway'), value=bytearray(b'traffic_signals'))], latitude=47.5145367, longitude=19.0385427)]

In [33]:
# key = assaas[1].tags[i][0]
# value = assaas[1].tags[i][1]

def func_for_string_search(string):
    def regex_filter(x):
        number_of_tags = len(x)
        flag = 0
        for i in range(0,number_of_tags):
            if ((string in x[i][0].decode("utf-8") ) or (string in x[i][1].decode("utf-8"))):
                flag +=1

        if flag > 0:
            return True
        else:
            return False

    filter_udf = udf(regex_filter, BooleanType())
    
    return filter_udf






In [34]:
f1 = func_for_string_search('taxi')

In [35]:
assaas = node_nonempty.select('id','tags','latitude','longitude').filter(f1('tags')).collect()


In [36]:
def get_lat_long(df):
    list1 = []
    for x in df:
        list1.append([x.id,x.latitude , x.longitude]) 
    return list1

In [37]:
def add_markers(m,list_of_lat_and_long_and_ID):
    for x in list_of_lat_and_long_and_ID:
        folium.Marker(location=[x[1],x[2]],popup=str(x[0])+','+str(x[1])+','+str(x[2]),size = 0.1).add_to(m)

In [38]:
l1 = get_lat_long(assaas)

In [39]:
import folium
import os


In [1]:
m = folium.Map(
    world_copy_jump=False,location=[47.5145367, 19.0385427],
    no_wrap=False,zoom_start=7)

add_markers(m,l1)

m

NameError: name 'folium' is not defined

In [41]:
#TRAINS

# f2 = func_for_string('railway')

def regex_filter(x):
    number_of_tags = len(x)
    flag = 0
    for i in range(0,number_of_tags):
        if (('railway' in x[i][0].decode("utf-8") ) and ('station' in x[i][1].decode("utf-8"))):
            flag +=1

    if flag > 0:
        return True
    else:
        return False

f2 = udf(regex_filter, BooleanType())



train_df = node_nonempty.select('id','tags','latitude','longitude').filter(f2('tags')).collect()



In [42]:
train_lat_long = get_lat_long(train_df)

In [43]:
len(train_lat_long)

954

In [44]:
m = folium.Map(
    world_copy_jump=False,location=[47.5145367, 19.0385427],
    no_wrap=False,zoom_start=7)

add_markers(m,train_lat_long)
m

In [45]:
def func_for_string_match(string1,string2):
    def regex_filter(x):
        number_of_tags = len(x)
        flag = 0
        for i in range(0,number_of_tags):
            if ((string1 in x[i][0].decode("utf-8") ) and (string2 in x[i][1].decode("utf-8"))):
                flag +=1

        if flag > 0:
            return True
        else:
            return False

    filter_udf = udf(regex_filter, BooleanType())
    
    return filter_udf



In [46]:
f3 = func_for_string_match('amen','fuel')
train_df = node_nonempty.select('id','tags','latitude','longitude').filter(f3('tags')).collect()
train_lat_long = get_lat_long(train_df)
m = folium.Map(
    world_copy_jump=False,location=[47.5145367, 19.0385427],
    no_wrap=False,zoom_start=7)

add_markers(m,train_lat_long)
m

In [47]:
f3 = func_for_string_search('amen')
train_df = node_nonempty.select('id','tags','latitude','longitude').filter(f3('tags')).collect()
train_lat_long = get_lat_long(train_df)

In [48]:
f3 = func_for_string_search('amen')
train_df = node_nonempty.select('id','tags','latitude','longitude').filter(f3('tags'))
train_lat_long = get_lat_long(train_df)


# df.withColumn('nb_customer',f.size(df.customers)).show()

# filter_udf = udf(regex_filter, BooleanType())

# train_df.withColumn('x_tag',f.size(df.customers)).show()

In [49]:
# train_df.withColumn('x_tag',train_df.latitude).show()
train_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: binary (nullable = true)
 |    |    |-- value: binary (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [50]:
#struct<key:binary,value:binary>
from pyspark.sql.types import *

string = 'amen'
def tag_extractor_helper(x):
    number_of_tags = len(x)
#     print(number_of_tags)
    flag = 0
    for i in range(0,number_of_tags):
        print(type(i))
        if (string in x[i][0].decode("utf-8")):
            return x[i][1].decode("utf-8")
    return x[0][1]
#     return []
           
# schema = StructType([StructFieldBinaryType(), True)])
tag_extractor_udf = udf(tag_extractor_helper, StringType())

# trai

In [51]:
new = train_df.withColumn('extract_tag',tag_extractor_udf(train_df.tags))


In [52]:
train_df = new.select('id','tags','latitude','longitude').filter(new.extract_tag == 'restaurant').collect()


In [54]:
train_lat_long = get_lat_long(train_df)
print (len(train_lat_long))

4657


In [56]:
way.take(5)

[Row(id=3175810, version=7, timestamp=1531865296000, changeset=60814610, uid=408450, user_sid=bytearray(b'B\xc3\xa1thoryP\xc3\xa9ter'), tags=[Row(key=bytearray(b'highway'), value=bytearray(b'residential')), Row(key=bytearray(b'name'), value=bytearray(b'Honv\xc3\xa9d utca'))], nodes=[Row(index=0, nodeId=15231786), Row(index=1, nodeId=1310148452), Row(index=2, nodeId=1310021025), Row(index=3, nodeId=1310020985), Row(index=4, nodeId=5766745506), Row(index=5, nodeId=1310020988), Row(index=6, nodeId=1310021030), Row(index=7, nodeId=1310021038), Row(index=8, nodeId=1310021044), Row(index=9, nodeId=1309949162), Row(index=10, nodeId=1309949099), Row(index=11, nodeId=1282709263), Row(index=12, nodeId=5766742402), Row(index=13, nodeId=1234760030)]),
 Row(id=3175943, version=3, timestamp=1366287135000, changeset=15772683, uid=408450, user_sid=bytearray(b'B\xc3\xa1thoryP\xc3\xa9ter'), tags=[Row(key=bytearray(b'highway'), value=bytearray(b'residential')), Row(key=bytearray(b'name'), value=bytearray